# Finetune GPT-2 on wiki-text

In this Lab, we are using a series of library from Hugging Face (i.e. tranformers, datasets, peft). You may need to go through the document of these library to learn the usage. (Hint: you may use the imported contents in the code cell below, other contents is not necessary for this lab)

In [2]:
# # for google colab
# !pip install transformers
# !pip install datasets
# !pip install peft

In [3]:
import torch
print(torch.__version__)

2.5.0+cu124


In [4]:
import os
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from datasets import load_dataset

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

from torch.utils.data import DataLoader
import torch.nn as nn

cuda


## Lab 2(a) Generate text with GPT2

Using the API provided by hugging face, we can easily load the pre-trained GPT2 model and generate text. (GPT2 is a early generative model, the quality of the generated text is not as good as the later model like GPT3.)

In [5]:
import os

# Print the value of HF_DATASETS_CACHE
print(f"HF_DATASETS_CACHE: {os.getenv('HF_DATASETS_CACHE')}")
os.environ['HF_DATASETS_CACHE'] = "D:\\MIDS\\Deep_Learning\\HW3\\resources"
print(f"HF_DATASETS_CACHE: {os.getenv('HF_DATASETS_CACHE')}")

HF_DATASETS_CACHE: None
HF_DATASETS_CACHE: D:\MIDS\Deep_Learning\HW3\resources


In [6]:
#Memory Issues, manually set cache dir so I can see space available.
os.environ['HF_DATASETS_CACHE'] = "D:\\MIDS\\Deep_Learning\\HW3\\resources"

# your code here: load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:

def generate_text(model, tokenizer, prompt, max_length):

    # your code here: tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    # your code here: generate token using the model
    gen_tokens = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, return_dict_in_generate=True)
    generated_ids = gen_tokens['sequences']
    # your code here: decode the generated tokens
    gen_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(gen_text)

generate_text(model, tokenizer, "GPT-2 is a langugae model based on transformer developed by OpenAI", 100)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GPT-2 is a langugae model based on transformer developed by OpenAI. It is a simple, fast, and scalable model of the human brain. It is based on the concept of the "brain as a machine".

The model is based on the concept of the "brain as a machine". The model is based on the concept of the "brain as a machine". The model is based on the concept of the "brain as a machine". The model is based on the


## Lab 2(b) Prepare dataset for training

Please fill the code cell below to download the dataset and prepare the dataset for finetuning.


In [8]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# your code here: load the dataset
dataset = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")

# Print the size of the dataset in bytes
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [9]:

# get 10% of dataset
dataset_train = dataset["train"].select(range(len(dataset["train"]) // 10))
dataset_valid = dataset["validation"].select(range(len(dataset["validation"]) // 10))
print(dataset_valid.shape)
print(dataset_train.shape)
del dataset #drop the full dataset from memory since we have memory issues later


(376, 1)
(180135, 1)


In [10]:

# your code here: implement function that tokenize the dataset and set labels to be the same as input_ids
def tokenize_function(examples,tokenizer=tokenizer):
    import torch #workers may not have torch in scope
    tokenized = tokenizer(examples["text"],padding='max_length', truncation=True)
    tokenized["labels"] = torch.tensor(tokenized["input_ids"]).clone()
    return tokenized


# your code here: tokenize the dataset (you may need to remove columns that are not needed)
tokenized_datasets_train = dataset_train.map(tokenize_function, batched=True, remove_columns=["text"],num_proc=3)
tokenized_datasets_valid = dataset_valid.map(tokenize_function, batched=True, remove_columns=["text"],num_proc=3)


tokenized_datasets_train.set_format("torch")
tokenized_datasets_valid.set_format("torch")

# your code here: create datacollator for training and validation dataset
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)#base arguments, could be wrong

train_dataloader = DataLoader(tokenized_datasets_train, shuffle=True, batch_size=4, collate_fn=data_collator)
valid_dataloader = DataLoader(tokenized_datasets_valid, batch_size=4, collate_fn=data_collator)

# Test the DataLoader
for batch in train_dataloader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['labels'].shape)
    break

print("DataLoader is working correctly!")

Map (num_proc=3):   0%|          | 0/180135 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/376 [00:00<?, ? examples/s]

torch.Size([4, 1024])
torch.Size([4, 1024])
torch.Size([4, 1024])
DataLoader is working correctly!


## Lab 2(c) Evaluate perplexity on wiki-text

Before finetuning, we evaluate the pre-trained GPT2 model on the wiki-text dataset. The perplexity is a common metric to evaluate the performance of language model. The lower the perplexity, the better the model. To compute the perplexity in practice, we use the formula as follows, which is a transformation of the formula in class:
$PP(W) = \exp\left(-\frac{1}{N} \sum_{i=1}^{N} \log P(w_i|\text{context})\right)$

In [13]:
def evaluate_perplexity(model, dataloader):
    model.eval()
    total_loss = 0
    total_length = 0
    loss_fn = nn.CrossEntropyLoss(reduction='sum')

    with torch.no_grad():
        for batch in dataloader:
            # your code here: get the input_ids, attention_mask, and labels from the batch
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # your code here: forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            
            # your code here: calculate the loss
            loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            
            total_loss += loss.item()
            total_length += attention_mask.sum().item()

    # Calculate perplexity
    perplexity = torch.exp(torch.tensor(total_loss / total_length))
    
    return perplexity.item()
    

perplexity = evaluate_perplexity(model, valid_dataloader)
print(f"Initial perplexity: {perplexity}")

Initial perplexity: 42.9958610534668


## Lab 2(d) Fine-tune GPT2 on wiki-text



In [14]:
#clear cache
torch.cuda.empty_cache()

In [ ]:

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    prediction_loss_only=True,
    # your code here: report validation and training loss every epoch
)

# your code here: create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,
)

trainer.train()
trainer.save_model()



  0%|          | 0/67551 [00:00<?, ?it/s]

# Test fine-tuned model

In [ ]:
# your code here: load the fine-tuned model
model_finetuned = 
perplexity = evaluate_perplexity(model_finetuned, valid_dataloader)
print(f"fine-tuned perplexity: {perplexity}")

# Generate some text using the fine-tuned model

In [ ]:
# load the fine-tuned model

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# generate text
generate_text(
    model_finetuned,
    tokenizer,
    "GPT-2 is a langugae model based on transformers developed by OpenAI",
    100,
)

## Lab 2(e) Parameter efficient fine-tuning (LoRA)

finetune the base gpt model through LoRA

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# your code here: load GPT2 model and add the lora adapter
model_lora = 


training_args = TrainingArguments(
    output_dir="./gpt2-lora-wikitext-2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    prediction_loss_only=True,
)

# your code here: set trainer and train the model
trainer = 

ppl = evaluate_perplexity(model_lora, valid_dataloader)
print(f"Perplexity after lora finetuning: {ppl}")


# Evaluate lora fine-tuned model on wiki-text

compare the text generated by the fully fine-tuned model and LoRA fine-tuned model and the pre-trained model. Do you see any difference in the quality of the generated text? Try to explain why. (Hint: trust your result and report as it is.)

In [ ]:
generate_text(
    model_lora,
    tokenizer,
    "GPT-2 is a langugae model based on transformers developed by OpenAI",
    100,
)

Compare the perplexity of the fully fine-tuned model and LoRA fine-tuned model. Do you see any difference in the perplexity? Try to explain why. 

In [ ]:
ppl = evaluate_perplexity(model_lora, valid_dataloader)

print(f"Perplexity after lora finetuning: {ppl}")